In [2]:
% matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_path= "/Users/Chris/Downloads/"
df = pd.read_csv(data_path+'HR_comma_sep.csv')

Let's start with the high level visualizations. I always like to use sns.pairplot to get the highest level visualization of my data.

The first look isn't always pretty, but it helps us get to know the distributions of the data, and helps us look for any interesting relationships

In [ ]:
sns.pairplot(data=df)
plt.show()

In [ ]:
for column in df.columns:
    if column != 'left':
        try:
            sns.jointplot('left', column, data = df)
            plt.title(column)
            plt.show()
        except:
            sns.countplot(x=column, hue='left', data=df)
            plt.show()

In [ ]:
# Checking out sales vs. leaving or not.
sns.countplot(x='sales', hue='left', data=df)
plt.show()

In [ ]:
# Salary vs. Leaving
# Looks like a high compensation prevents employees from leaving. A surprise to nobody.
sns.countplot(x='salary',hue='left', data=df)
plt.show()

In [ ]:
# Satisfaction Level and Salary are positively correlated, but not that strongly.
# The average still seems to be around 60% for each though
sns.barplot(x='salary', y='satisfaction_level', data=df)

In [ ]:
# But on average, those with a low salary are more likely to leave.
sns.barplot(x='salary', y='left', data=df)

In [ ]:
# Similar
sns.barplot(x='left', y='last_evaluation', data=df)

In [ ]:
# Another question, how does last_evaluation relate to
# satisfaction_level.
sns.jointplot(x='last_evaluation', y='satisfaction_level', data=df,
             kind='reg')
# While statistically significant in isolation, we still don't know very much
# about the relationship

The above chart plots give us a couple indicators for left. The pearson correlation coefficients can give a small clue as to the relationships between each variable, and the jointplots.

1. The more hours worked by an individual, the more likely the individual is to leave.

2. Receiving a promotion in the last five years decreases the likelihood of leaving

3. Satisfaction Level is negatively correlated

An important question to answer in these sorts of business cases is "What are the factors that are determining whether an employee leaves." We don't want to just predict whether an employee is going to leave via a black box. This is an inappropriate response to the issue. What we actually want is some way to know what factors are contributing the most to an employee leaving.

<h3>Constructing a Model</h3>
What we will need to do is deal with some of the categorical variables. We need to one-hot encode non-orderable categorical variables. For example, the sales category must have a column per sales category, since there is no order implied in the categories.  However salary should be coded as -1, 0, +1 for low, medium, high respectively.

I prefer to use Pandas to do the recoding, because it is super easy, and I love dataframes 

In [4]:
# Creating the dummies for sales
sales_dummies = pd.get_dummies(df['sales'])

df = pd.concat([df, sales_dummies], axis=1)
df.drop('sales', inplace = True, axis=1)

def recode_sales(row):
    if row == 'low':
        return -1
    elif row == 'medium':
        return 0
    else:
        return 1
    
df['salary'] = df['salary'].apply(recode_sales)

<h3>Evaluating Initial Model Options</h3>
The choice of model in this case is partially dictated by business needs. Ideally we would like some causal relationship, but we don't have the data to get that. We could do with some sort of inference, like through a properly applied Logistic Regression. Another option is a Random Forest, which gives "Variable Importance." If we were to give a presentation to management (which we won't obviously), a decision tree could be an easily communicable way to management, especially if it had comparable predictive performance to the random forest.


<h3>Logistic Regression</h3>
Since we are dealing with a binary model, a logistic regression is a linear classifier that allows for a flexible enough fit (with basis transformations) while allowing for interpretability. 

In [5]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

x = np.array(df.drop('left', axis=1))
y = np.array(df['left'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
x, y, train_size=0.8)

from sklearn.model_selection import cross_val_score
logit_score = cross_val_score(logit, X_train, Y_train, cv=10)
print(logit_score.mean()) # This is an embarassingly bad score

0.794729275275


In [6]:
# Let's investigate this score a little more closely
# This might be wrong
logit_fit = logit.fit(X_train, Y_train)
logit_coef = logit_fit.coef_
names_list = []
for i in df.drop('left',axis=1).columns:
    names_list.append(i)
    

for i in range(len(names_list)):
    print(names_list[i], logit_coef[0][i])

satisfaction_level -4.06364676387
last_evaluation 0.701241758085
number_project -0.330955048032
average_montly_hours 0.0046951926699
time_spend_company 0.256482194405
Work_accident -1.5513677389
promotion_last_5years -1.23628774989
salary -0.67981029325
IT -0.0319042815875
RandD -0.543508814657
accounting 0.0784802786253
hr 0.294209940887
management -0.479169200253
marketing 0.0543873081108
product_mng -0.0816582471712
support 0.136135577679
technical 0.133632537671


In [7]:
# Now sci-kit learn isn't actually the right tool to use for this.
# This is more for a statsmodels thing
import statsmodels.api as sm
sm_logit = sm.Logit(df['left'], df.drop('left',axis=1))
sm_logit_result = sm_logit.fit()

sm_logit_result.summary()
# This gives us some information as to the significance of the variables,
# and it seems that satisfaction level is the largest determinant

Optimization terminated successfully.
         Current function value: 0.429809
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   left   No. Observations:                14999
Model:                          Logit   Df Residuals:                    14982
Method:                           MLE   Df Model:                           16
Date:                Fri, 07 Apr 2017   Pseudo R-squ.:                  0.2169
Time:                        13:39:51   Log-Likelihood:                -6446.7
converged:                       True   LL-Null:                       -8232.3
                                        LLR p-value:                     0.000
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
satisfaction_level       -4.1853      0.092    -45.449      0.000        -4.366    -4.005
last_evaluation           0.6501      0.141      4.600      0.000         0.373     0.927
number_project           -0.3141      0.021    -14.838      0.000        -0.356    -0.273
average_montly_hours      0.0041      0.000      8.538      0.000         0.003     0.005
time_spend_company        0.2565      0.015     17.350      0.000         0.228     0.285
Work_accident            -1.5352      0.090    -17.146      0.000        -1.711    -1.360
promotion_last_5years    -1.4487      0.258     -5.623      0.000        -1.954    -0.944
salary                   -0.6758      0.037    -18.056      0.000        -0.749    -0.602
IT                       -0.1622      0.088     -1.852      0.064        -0.334     0.009
RandD                    -0.5639      0.117     -4.810      0.000        -0.794    -0.334
accounting                0.0118      0.101      0.117      0.907        -0.186     0.210
hr                        0.2597      0.100      2.608      0.009         0.065     0.455
management               -0.5206      0.134     -3.874      0.000        -0.784    -0.257
marketing                -0.0077      0.101     -0.076      0.939        -0.205     0.190
product_mng              -0.1417      0.098     -1.440      0.150        -0.334     0.051
support                   0.0635      0.069      0.926      0.355        -0.071     0.198
technical                 0.0822      0.064      1.282      0.200        -0.043     0.208
=========================================================================================
"""


### Interpreting Regression Results
Our first stab at Logistic Regression did not produce a great prediction score through scikit learn, however we were able to examine the coefficients and their significance through sci-kit learn and statsmodels. Note that there are slight differences between the coefficient estimates between the two models, this is because sci-kit learns package uses regularization by default in their models, whereas statsmodels does not.

### Bias in Logistic Regression

We are running a very simple model, where each of the coefficients are linear, and of one degree. This is not particularly helpful, as one could imagine, since the complexity of the relationships in reality are likely not linear.

One way to get a little more of a complex model is by employing basis expansions of our data. By adding higher-order polynomials, we can increase complexity. The tradeoff is the risk of overfitting. Combining Variable Selection with high-order basis transformations could allows us to increase the flexibility where it's warranted.


In [8]:
# Let's go back to the original training dataframe

df_train = df.drop('left', axis=1)



In [9]:
# A function that performs basis expansion of orders and interactions in your dataframe
def basis_expansion(data_frame, order=2, interactions = False):
    # Creates empty dataframe
    # We will append our new columns into the df, and then
    # concatenate our new frame to the original
    new_df = pd.DataFrame()
    relevant_col_list = []
    
    
    def check_if_relevant(df):
        for col in df.columns:
            unique_vals = list(df[col].unique())
            if not (len(unique_vals) == 2 and 0 in unique_vals and 1 in unique_vals):
                relevant_col_list.append(col)
            
    check_if_relevant(data_frame)
    for exponent in (range(1,order)):
        exponent += 1 # For each exponent, add columns
        for col in relevant_col_list:
       
            new_df[col+'**'+str(exponent)] = data_frame[col]**exponent
    
    if interactions:
        for col1 in data_frame.columns:
            for col2 in data_frame.columns:
                if col1 != col2:
                    new_df[col1+"*"+col2] = data_frame[col1]*data_frame[col2]
    new_dataframe = pd.concat([data_frame, new_df], axis=1)
    return new_dataframe

In [10]:
# Now we have a basis expansion, with likely too many terms
df_basis_expand = basis_expansion(df_train, order=3, interactions=True)
feature_names = [df_basis_expand.columns]

In [11]:
# Creating the sklearn logistic regression
basis_logit = LogisticRegression()
xb_train, xb_test, yb_train, yb_test = train_test_split(np.array(df_basis_expand), np.array(df['left']), train_size = 0.8)

In [12]:
from sklearn.model_selection import cross_val_score
bx_logit_score = cross_val_score(basis_logit, xb_train, yb_train, cv=10)
print(bx_logit_score.mean()) # This is worse than before

0.787814538876


### Recall that sklearn's logistic regression has regularization built into it.

Because of this, we can play with their regularization parameter.


... at this point I'm pursuing Logistic Regression out of stubbornness

In [15]:
# Testing out as many parameters for regularization as possible
regularization_dict = {}

for c in np.arange(0.01, 1.01, 0.03):
    basis_logit_c = LogisticRegression(C=c, penalty='l1')
    bx_logit_score_c = cross_val_score(basis_logit_c, xb_train, yb_train, cv=10)
    regularization_dict[c] = bx_logit_score_c.mean()

In [ ]:
cross_val_score(LogisticRegression(C=0.001), xb_train, yb_train, cv=10).mean()

In [16]:
regularization_dict

{0.01: 0.90648594952727957,
 0.040000000000000001: 0.92674539964958313,
 0.069999999999999993: 0.94174457106104481,
 0.099999999999999992: 0.94574519727675743,
 0.13: 0.94741151712605354,
 0.16: 0.94974443437113509,
 0.19: 0.95099471226484644,
 0.22: 0.95091144854961696,
 0.25: 0.95099485138531337,
 0.28000000000000003: 0.95182832360763214,
 0.31: 0.95174505977666191,
 0.33999999999999997: 0.95024512916328407,
 0.37: 0.95099672517133682,
 0.40000000000000002: 0.95257832349189131,
 0.42999999999999999: 0.95099485115383187,
 0.45999999999999996: 0.95007846249661743,
 0.48999999999999999: 0.95241193460319518,
 0.52000000000000002: 0.95266172632758772,
 0.55000000000000004: 0.95216193471893606,
 0.57999999999999996: 0.95307860138560285,
 0.60999999999999999: 0.95291193471893609,
 0.64000000000000001: 0.95232860126986196,
 0.66999999999999993: 0.95099610069173646,
 0.69999999999999996: 0.95157811486906119,
 0.72999999999999998: 0.95332867088796591,
 0.76000000000000001: 0.9532452680522695,


### Results
95% on a cross-validation score is very impressive for an order two model!
The next step is to evaluate the coefficients, and run it through statsmodel's logistic regression, to get confidence intervals. Further fine tweaking could possibly allow for interpretable coefficients. 

P.S the highest Cross-Validation score is with a C value of 0.82

NameError: name 'regularization_dict' is not defined